In [ ]:
# PS-9, PS-10, PS-11, PS-16 : Object Detection 

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
import os

print(f"Using TensorFlow version: {tf.__version__}")

# --- Setup: Define Paths and Image Parameters ---
IMG_WIDTH, IMG_HEIGHT = 224, 224 # VGG16 standard size
BATCH_SIZE = 32
DATA_DIR = r"LP-IV-datasets\Object Detection(Ass6)\caltech-101-img"
WEIGHTS_PATH = r"LP-IV-datasets\Object Detection(Ass6)\vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
# Check if files exist
if not os.path.exists(DATA_DIR):
    print(f"Error: Dataset folder '{DATA_DIR}' not found.")
    exit()
if not os.path.exists(WEIGHTS_PATH):
    print(f"Error: Weights file '{WEIGHTS_PATH}' not found.")
    exit()

# --- Setup: Load and Preprocess Data ---
# We use ImageDataGenerator to load from folders.
# We also split the data into 80% training and 20% validation.
train_datagen = ImageDataGenerator(
    rescale=1./255,            # Normalize pixel values
    validation_split=0.2,      # Hold back 20% for validation
    shear_range=0.2,           # Add some data augmentation
    zoom_range=0.2,
    horizontal_flip=True
)

# Training data generator
train_generator = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training' # Set as training data
)

# Validation data generator
validation_generator = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation' # Set as validation data
)

# Get the number of classes (subfolders)
num_classes = train_generator.num_classes
print(f"Found {num_classes} classes (subfolders) in '{DATA_DIR}'.")

# --- a. Load in a pre-trained CNN model ---
print("\n--- a. Loading pre-trained VGG16 model ---")
# 1. Initialize VGG16 with the correct input shape
#    We set 'weights=None' because we will load them from your file
base_model = VGG16(
    weights=None, 
    include_top=False, # Do not include the final classifier
    input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)
)

# 2. Load the weights from your specific file
base_model.load_weights(WEIGHTS_PATH)
print("Model and custom weights loaded successfully.")


# --- b. Freeze parameters in lower convolutional layers ---
print("\n--- b. Freezing base model layers ---")
# We freeze all layers in the base model so we only train our new classifier
for layer in base_model.layers:
    layer.trainable = False


# --- c. Add custom classifier ---
print("\n--- c. Adding new custom classifier ---")
# 1. Get the output of the base model
x = base_model.output

# 2. Add our new layers
x = Flatten()(x) # Flatten the 3D features to 1D
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x) # Dropout for regularization
predictions = Dense(num_classes, activation='softmax')(x) # Output layer

# 3. Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

print("New model created with VGG16 base and custom head.")
model.summary()


# --- d. Train classifier layers on training data ---
print("\n--- d. Training the custom classifier (Phase 1) ---")
# Compile the model for the first round of training
model.compile(
    optimizer=Adam(learning_rate=1e-4), # Use Adam with a moderate learning rate
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model (e.g., for 10 epochs)
# We only train the new Dense/Dropout layers
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)


# --- e. Fine-tune hyperparameters and unfreeze layers ---
print("\n--- e. Fine-tuning the model (Phase 2) ---")
# Now we unfreeze the top block of VGG16 (block5) to fine-tune it
print("Unfreezing 'block5' of VGG16...")
for layer in base_model.layers:
    if layer.name.startswith('block5'):
        layer.trainable = True

# Re-compile the model with a VERY low learning rate for fine-tuning
# This prevents destroying the learned weights
model.compile(
    optimizer=Adam(learning_rate=1e-5), # Must be very low!
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Continue training (fine-tuning) for 10 more epochs
history_fine_tune = model.fit(
    train_generator,
    epochs=20, # Continue from epoch 10 to 20
    initial_epoch=history.epochs, # Start where we left off
    validation_data=validation_generator
)

print("\n--- Model training complete ---")

# Evaluate the final model
print("\nFinal Model Evaluation:")
final_loss, final_accuracy = model.evaluate(validation_generator)
print(f"Final Validation Loss: {final_loss:.4f}")
print(f"Final Validation Accuracy: {final_accuracy * 100:.2f}%")

Using TensorFlow version: 2.10.1
Found 7357 images belonging to 102 classes.
Found 1788 images belonging to 102 classes.
Found 102 classes (subfolders) in 'LP-IV-datasets\Object Detection(Ass6)\caltech-101-img'.

--- a. Loading pre-trained VGG16 model ---
Model and custom weights loaded successfully.

--- b. Freezing base model layers ---

--- c. Adding new custom classifier ---
New model created with VGG16 base and custom head.
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling